In [14]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms

class MLP(nn.Module):
  def __init__(self,input,output):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(input, 16),
      nn.ReLU(),
      nn.Linear(16,20),
      nn.ReLU(),
      nn.Linear(20,output),
      nn.Softmax(dim=1)
    )
  def forward(self,x):
    return self.layers(x)

In [15]:
import torch

class DataSet(torch.utils.data.Dataset):
    def __init__(self,data,labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        return self.data[idx], self.labels[idx]

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split

data = np.genfromtxt('iris.data',delimiter=',')
data = np.delete(data,-1,axis=1)
classes = np.genfromtxt('iris.data',delimiter=',', dtype=str,usecols=[-1])
classes = np.unique(classes,return_inverse=1)[1]

data_treino, data_teste, classe_treino, classe_teste = train_test_split(data,classes,test_size=0.2,stratify=classes,random_state=1)

dataset = DataSet(data_treino,classe_treino)
data_train = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

dataset = DataSet(data_teste,classe_teste)
data_test = DataLoader(dataset, batch_size=len(data_teste), shuffle=True, num_workers=0)

In [17]:
mlp = MLP(4,3)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

In [18]:
epochs = 1000

mlp.train(True)
for epoch in range(epochs):
    print(f'Starting epoch {epoch+1}')

    current_loss = 0.0
    total_samples = 0

    for data in data_train:
        x,y = data
        x = x.to(torch.float32)
        optimizer.zero_grad()
        output = mlp(x)
        loss = loss_function(output,y)
        loss.backward()
        optimizer.step()
        current_loss += loss.item()
        total_samples += len(x)
    
    print(f'Loss: {current_loss/total_samples}')

print('Training process has finished.')

Starting epoch 1
Loss: 0.037070663770039876
Starting epoch 2
Loss: 0.03703748385111491
Starting epoch 3
Loss: 0.037044999996821086
Starting epoch 4
Loss: 0.03697333534558614
Starting epoch 5
Loss: 0.03697166740894318
Starting epoch 6
Loss: 0.03699002861976623
Starting epoch 7
Loss: 0.037013996640841165
Starting epoch 8
Loss: 0.03703135947386424
Starting epoch 9
Loss: 0.03703126112620036
Starting epoch 10
Loss: 0.037000385920206706
Starting epoch 11
Loss: 0.036959709723790486
Starting epoch 12
Loss: 0.03687449991703033
Starting epoch 13
Loss: 0.03691970407962799
Starting epoch 14
Loss: 0.036983866492907205
Starting epoch 15
Loss: 0.03696958919366201
Starting epoch 16
Loss: 0.0368166834115982
Starting epoch 17
Loss: 0.03690665066242218
Starting epoch 18
Loss: 0.03687121570110321
Starting epoch 19
Loss: 0.03691820204257965
Starting epoch 20
Loss: 0.036778056621551515
Starting epoch 21
Loss: 0.03688806891441345
Starting epoch 22
Loss: 0.03675506114959717
Starting epoch 23
Loss: 0.036831532

In [19]:
from sklearn.metrics import classification_report

mlp.train(False)

for data in data_test:
    x_test,y_test = data
    x_test = x_test.to(torch.float32)
    y_pred = mlp(x_test)
    y_pred = torch.Tensor.argmax(y_pred, dim=1)
    y_pred = torch.Tensor.numpy(y_pred)
    y_test = torch.Tensor.numpy(y_test)

print(classification_report(y_test,y_pred,zero_division=1))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

